# 配置环境配置

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

# proj_dir = "/content/drive/MyDrive/Colab Notebooks"

proj_dir = "/home/ubuntu/git/DBMA"

# 安装包和配置OpenAI

In [ ]:
%pip uninstall agency-swarm -y
%pip install --upgrade --no-cache-dir git+https://github.com/dcstrange/agency-swarm.git@dbOps_Agency #AgencySwarm
%pip install -q gradio # agency 会话前端
#%pip install -q duckduckgo_search # Web searching

import os
secret_dir = proj_dir + "/secret-configs"
with open(secret_dir + "/OAI_API_KEY") as f:
    os.environ['OPENAI_API_KEY'] = f.read()

from agency_swarm import set_openai_key
set_openai_key(os.environ['OPENAI_API_KEY'])

# Or
# from getpass import getpass
# set_openai_key(getpass("Please enter your openai key: "))

# 🔨Pseudo-database Environment
💡This is a pseudo-database environment implemented by manual input and output just for a Demo version.
This interface delegates specific tasks requested by other Agents to be performed manually.
This API will be replaced by a real Agent in Dev version, which is responsible for interacting with the database environment.

这是一个通过手动输入和输出实现的伪数据库环境，仅用于Demo版本。此接口委托其他代理要求手动执行的特定任务。这个 API 在Dev版本中会被一个真正的代理所替代，代理负责与数据库环境进行交互。

In [15]:
from agency_swarm import BaseTool
from pydantic import Field
from db_pseudo_env.db_task_api import Assign_DB_Task

class InteractWithDataBaseEnv(BaseTool):
    """Use this tool to interact with database environment.
    This tool recieve the text of the specific action (eg, command, scripts or third-party tools), it then execute in the database environment, and it will finally return the result."""

    specific_task: str = Field(..., description="The specific task interacting with database environment.")


    # This code will be executed if the agent calls this tool
    def run(self):
      return str(Assign_DB_Task(self.specific_task)['response'])


In [16]:
agent_db_env_instructions = """ #Instructions for Database Environment Agent

You are a highly specialized GPT focused on SQL database operations for professional database administrators and developers.
You understand specific intents recieved from other agents and generates appropriate database actions using commands, scripts, or third-party tools.
Your primary objective is to interact effectively with SQL databases, providing accurate and advanced technical solutions.
you focuses on returning precise results and analyzing them for technical users.
You maintains a professional and straightforward tone, prioritizing technical accuracy and practicality over personalization, ensuring that its responses are detailed, relevant, and valuable to database professionals.
### You have to response to agents who sent you specific intent with the result of the Function Output.
"""


In [17]:
from agency_swarm import Agent

agent_db_env = Agent(name="Database Environment",
                     tools=[InteractWithDataBaseEnv],
                     description="responsible for taking specific tasks and executing them in the database environment (including databases, monitoring systems, etc.), planning and executing specific actions.",
                     instructions=agent_db_env_instructions,
                     files_folder=None)


# CEO Agent


In [10]:
ceo_instructions = """# Instructions for CEO Agent

- Ensure that proposal is send to the user before proceeding with task execution.
- Delegate tasks to appropriate agents, ensuring they align with their expertise and capabilities.
- Clearly define the objectives and expected outcomes for each task.
- Provide necessary context and background information for successful task completion.
- Maintain ongoing communication with agents until complete task execution.
- Review completed tasks to ensure they meet the set objectives.
- Report the results back to the user."""

In [11]:
from agency_swarm import Agent

ceo = Agent(name="CEO",
            description="Responsible for client communication, task planning and management.",
            instructions=ceo_instructions, # can be a file like ./instructions.md
            files_folder=None,
            tools=[])

# Virtual Assistant

### Importing tools from langchain example.  
You can skip these and remove them from va agent below.

In [ ]:
!pip install langchain==0.0.318 &> /dev/null

In [ ]:
from langchain.utilities.zapier import ZapierNLAWrapper
from langchain.agents.agent_toolkits import ZapierToolkit
import os
from langchain.tools import format_tool_to_openai_function
from langchain.tools.zapier.tool import ZapierNLARunAction

# https://nla.zapier.com/docs/authentication/
os.environ["ZAPIER_NLA_API_KEY"] = getpass("Your Zapier NLA Key: ")

In [ ]:
from agency_swarm import BaseTool
from pydantic import Field

actions = ZapierNLAWrapper().list()

class FindEmail(BaseTool):
    """Use this tool to find an email in user's mailbox"""
    search_string: str = Field(..., description="The search phrase you want to use to find a relevant email.")

    def run(self):
      action = next(
          (a for a in actions if a["description"].startswith("Gmail: Find Email")), None
      )
      return str(ZapierNLARunAction(
              action_id=action["id"],
              zapier_description=action["description"],
              params_schema=action["params"],
          ).run(self.search_string))

class DraftEmail(BaseTool):
    """Use this tool to draft a response email"""
    instructions: str = Field(..., description="Clearly outline in natural language the content of the email you need to draft and the address of the recepient.")

    def run(self):
        action = next(
            (a for a in actions if a["description"].startswith("Gmail: Create Draft Reply")), None
        )
        return str(ZapierNLARunAction(
                action_id=action["id"],
                zapier_description=action["description"],
                params_schema=action["params"],
            ).run(self.instructions))

### Custom tools

In [ ]:
from duckduckgo_search import DDGS
from agency_swarm.util.oai import get_openai_client

client = get_openai_client()


class SearchWeb(BaseTool):
    """Search the web with a search phrase and return the results."""

    phrase: str = Field(..., description="The search phrase you want to use. Optimize the search phrase for an internet search engine.")

    # This code will be executed if the agent calls this tool
    def run(self):
      with DDGS() as ddgs:
        return str([r for r in ddgs.text(self.phrase, max_results=3)])

class GenerateProposal(BaseTool):
    """Generate a proposal for a project based on a project brief. Remember that user does not have access to the output of this function. You must send it back to him after execution."""
    project_brief: str = Field(..., description="The project breif to generate a proposal for.")

    def run(self):
        completion = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
              {"role": "system", "content": "You are a professional proposal drafting assistant. Do not include any actual technologies or technical details into proposal until specified in the project brief. Be short."},
              {"role": "user", "content": "Please draft a proposal for the ollowing project brief: " + self.project_brief}
            ]
          )

        return str(completion.choices[0].message.content)

In [ ]:
va_instructions = """### Instructions for Virtual Assistant

Your role is to assist users in executing tasks like below. If the task is outside of your capabilities, please report back to the user.

#### 1. Drafting Emails
   - **Understand Context and Tone**: Familiarize yourself with the context of each email. Maintain a professional and courteous tone.
   - **Accuracy and Clarity**: Ensure that the information is accurate and presented clearly. Avoid jargon unless it's appropriate for the recipient.

#### 2. Generating Proposals
   - **Gather Requirements**: Collect all necessary information about the project, including client needs, objectives, and any specific requests.

#### 3. Conducting Research
   - **Understand the Objective**: Clarify the purpose and objectives of the research to focus on relevant information.
   - **Summarize Findings**: Provide clear, concise summaries of the research findings, highlighting key points and how they relate to the project or inquiry.
   - **Cite Sources**: Properly cite all sources to maintain integrity and avoid plagiarism.
"""

In [ ]:
va = Agent(name="Virtual Assistant",
            description="Responsible for drafting emails, doing research and writing proposals.",
            instructions=va_instructions,
            files_folder=None,
            tools=[SearchWeb, FindEmail, DraftEmail, GenerateProposal])

# Developer Agent

In [ ]:
from agency_swarm.tools import BaseTool
from pydantic import Field, BaseModel
import subprocess
from typing import List


class ExecuteCommand(BaseTool):
    """Run any command from the terminal. If there are too many logs, the outputs might be truncated."""
    command: str = Field(
        ..., description="The command to be executed."
    )

    def run(self):
        """Executes the given command and captures its output and errors."""
        try:
            # Splitting the command into a list of arguments
            command_args = self.command.split()

            # Executing the command
            result = subprocess.run(
                command_args,
                text=True,
                capture_output=True,
                check=True
            )
            return result.stdout
        except subprocess.CalledProcessError as e:
            return f"An error occurred: {e.stderr}"

class File(BaseTool):
    """
    File to be written to the disk with an appropriate name and file path, containing code that can be saved and executed locally at a later time.
    """
    file_name: str = Field(
        ..., description="The name of the file including the extension and the file path from your current directory if needed."
    )
    body: str = Field(..., description="Correct contents of a file")

    def run(self):
        # Extract the directory path from the file name
        directory = os.path.dirname(self.file_name)

        # If the directory is not empty, check if it exists and create it if not
        if directory and not os.path.exists(directory):
            os.makedirs(directory)

        # Write the file
        with open(self.file_name, "w") as f:
            f.write(self.body)

        return "File written to " + self.file_name

class Program(BaseTool):
    """
    Set of files that represent a complete and correct program. This environment has access to all standard Python packages and the internet.
    """
    chain_of_thought: str = Field(...,
        description="Think step by step to determine the correct actions that are needed to implement the program.")
    files: List[File] = Field(..., description="List of files")

    def run(self):
      outputs = []
      for file in self.files:
        outputs.append(file.run())

      return str(outputs)

In [ ]:
dev_instructions = """# Instructions for AI Developer Agent

- Write clean and efficient Python code.
- Structure your code logically, with `main.py` as the entry point.
- Ensure correct imports according to program structure.
- Execute your code to test for functionality and errors, before reporting back to the user.
- Anticipate and handle potential runtime errors.
- Provide clear error messages for easier troubleshooting.
- Debug any issues before reporting the results back to the user."""

In [ ]:
from agency_swarm.tools import Retrieval, CodeInterpreter

dev = Agent(name="Developer",
            description="Responsible for running and executing Python Programs.",
            instructions=dev_instructions,
            files_folder=None,
            tools=[ExecuteCommand, Program])


# Create Agency

In [12]:
agency_manifesto = """# "VRSEN AI" Agency Manifesto

You are a part of a virtual AI development agency called "VRSEN AI"

Your mission is to empower businesses to navigate the AI revolution successfully."""

In [18]:
from agency_swarm import Agency

agency = Agency([
    ceo,
    [ceo, agent_db_env],
], shared_instructions=agency_manifesto)

Updating assistant... Database Environment


In [19]:
agency.demo_gradio(height=900)

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Text sent successfully, check the http://127.0.0.1:5000/task.
Reply: This requires a JOIN operation between the Employees and Departments tables on the DepartmentID column, along with a WHERE clause to filter for 'IT' in the DepartmentName column. 
The expected outcome is a list of names of employees in the IT department. 
{'response': "This requires a JOIN operation between the Employees and Departments tables on the DepartmentID column, along with a WHERE clause to filter for 'IT' in the DepartmentName column. \nThe expected outcome is a list of names of employees in the IT department. ", 'task_id': 'ef130b1f-a1a1-48a4-8039-11f7e0538dd2'}


In [ ]:
#%pip install gradio
import gradio as gr

def greet(name):
    return "Hello " + name + "!"

demo = gr.Interface(fn=greet, inputs="text", outputs="text")
demo.launch()